# ERCPy: 2D Gaussian fit demonstration notebook

#### by Martial Duchamp
Laboratory for in situ and operando nanoscopy
Nanyang Technological University
mduchamp@ntu.edu.sg

### Absract:
The notebook demonstrates usage of the 2D Gaussian function to fit HR (S-)TEM images

## Preamble:

#### Plotting outside the notebook to enable interactions with user:

In [1]:
%matplotlib qt4

In [2]:
import hyperspy.api as hs

In [3]:
import os
import scipy as sc
import numpy as np

In [4]:
import matplotlib.pyplot as plt

## Loading data:

In [5]:
path="/home/martial/Data/Divers_Project_NTU/Paloma/Test_fitting/"

In [6]:
file_name="9039.dm3"
specImg = hs.load(path+file_name)

#### Plotting:

In [7]:
#Crop spatially, if needed
specImg.crop(0, 1.,9.)
specImg.crop(1, 4.,12.)

In [8]:
specImg.plot()

#### Estimation of the local minima (or maxima)

In [11]:
# Copy of the specImg, where the local min/max will be stored
min_specImg = specImg.deepcopy()

In [12]:
def list_distance (centroids,dataPoints):
    #centroids = np.array([[3,44]])
    #dataPoints = np.array([[3,44],[5,15]])

    def size(vector):
        return np.sqrt(sum(x**2 for x in vector))

    def distance(vector1, vector2):
        return size(vector1 - vector2)

    def distances(array1, array2):
        return [[distance(vector1, vector2) for vector2 in array2] for vector1 in array1]

    return (distances(centroids, dataPoints))

In [9]:
    def lenght(vector):
        return np.sqrt(sum(x**2 for x in vector))
    
    def distance(vector1, vector2):
        return lenght(vector1 - vector2)
    
    def find_local_maxima(self,smooth_size, threshold, border_size,close_points):
        """Returns a map (array) of the image which is TRUE at positions 
        of a local maxima"""
        
        local_min_self= np.empty_like(self, dtype=bool)
        local_min_self[:,:]=False
        
        # TO DO: remove multiple entries to local maxima
        #smooth_self = sc.ndimage.filters.gaussian_filter(specImg.data, sigma=smooth_size)
        #local_min = sc.ndimage.maximum_filter(smooth_self, size = 0) == smooth_self
        #local_min[:,:] = False
        
        data_max = sc.ndimage.filters.maximum_filter(specImg.data, smooth_size)
        maxima = (specImg.data == data_max)
        data_min = sc.ndimage.filters.minimum_filter(specImg.data, smooth_size)
        diff = ((data_max - data_min) > threshold)
        maxima[diff == 0] = 0
        
        labeled, num_objects = sc.ndimage.label(maxima)
        slices = sc.ndimage.find_objects(labeled)
        x, y = [], []
        for dy,dx in slices:
            x_center = (dx.start + dx.stop - 1)/2
            x.append(x_center)
            y_center = (dy.start + dy.stop - 1)/2    
            y.append(y_center)
            
        #Delete close points
        x2, y2 = [], []
        for j in range (0,np.size(x)-1):
            notok = True
            for i in range (j+1,np.size(x)-1):
                d=list_distance(np.array([[x[j],y[j]]]),np.array([[x[i],y[i]]]))[0][0]
                if d<close_points:
                    notok = False
                    x[i]=(x[j]+x[i])/2
                    y[i]=(y[j]+y[i])/2
                    break
            if notok == True:
                x2.append(x[j])
                y2.append(y[j])
       
        for i in range (0,np.size(x2)):
            local_min_self[int(y2[i]),int(x2[i])]=True
  
        # delete borders:
        borders = np.ones_like(self, dtype=np.bool)
        borders[border_size:-border_size, border_size:-border_size] = False
        
        detected_peaks = np.logical_and(local_min_self, np.logical_not(borders))
        #plt.plot(x2,y2, 'ro')
        return detected_peaks

In [13]:
min_specImg.data=find_local_maxima(specImg.data,17,5000,100,15)
# The values has to be adjusted to get maxima/minima

In [14]:
min_specImg.plot()

In [ ]:
#remove the calibration, then the coordinates are given in pixel in the plot
min_specImg.axes_manager.show();
#Use the GUI to set scale=1 and origin=0 for both x and y

In [ ]:
#remove the calibration, then the coordinates are given in pixel in the plot
specImg.axes_manager.show();
#Use the GUI to set scale=1 and origin=0 for both x and y

In [15]:
#Make a np array with the coordinate of the minima positions
point_center=np.zeros((np.size(min_specImg.data.nonzero()[1]),2),dtype=int) #np.size(min_specImg.data.nonzero()[0])
point_center[:,0]=min_specImg.data.nonzero()[0][:];
point_center[:,1]=min_specImg.data.nonzero()[1][:];

Traceback (most recent call last):
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/backends/backend_qt5.py", line 519, in _draw_idle
    self.draw()
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/backends/backend_agg.py", line 433, in draw
    self.figure.draw(self.renderer)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/figure.py", line 1475, in draw
    renderer, self, artists, self.suppressComposite)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/image.py", line 141, in _draw_list_compositing_images
    a.draw(renderer)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/martial/anaconda3/lib/python3.5/site-package

In [ ]:
#point_center[np.shape(point_center)[0]-4,:]=[129,55];

#### Curve fitting

In [16]:
from scipy.optimize import curve_fit
import scipy.optimize as opt

In [17]:
def twoD_Gaussian(u, para):
    x = u[0]
    y = u[1]
    amplitude, xo, yo, sigma, offset = para
    xo = float(xo)
    yo = float(yo)    
    a = 1/(2*float(sigma)**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + a*((y-yo)**2)))    
    return g

In [18]:
#Define the twoD_Gaussian for fitting ONLY
def Gaussian_2D(u, amplitude, xo, yo, sigma, offset):
        x = u[0]
        y = u[1]
        xo = float(xo)
        yo = float(yo)    
        a = 1/(2*float(sigma)**2)
        g = offset + amplitude*np.exp( - (a*((x-xo)**2) + a*((y-yo)**2)))    
        return g.ravel()

def twoD_Gaussian_fit(exp_data):
    amplitude = np.std(exp_data) #- is minimun are fitted
    xo = np.shape(exp_data)[0]/2
    yo = np.shape(exp_data)[1]/2
    sigma = (np.shape(exp_data)[0]+np.shape(exp_data)[1])/10
    offset = np.mean(exp_data)
    initial_guess = (amplitude, xo, yo, sigma, offset)
    
    # Create x and y indices
    x = np.linspace(0, np.shape(exp_data)[0], np.shape(exp_data)[0])
    y = np.linspace(0, np.shape(exp_data)[1], np.shape(exp_data)[1])
    u = np.meshgrid(x, y) 

    popt, pcov = opt.curve_fit(Gaussian_2D, u, exp_data.ravel(), p0=initial_guess, maxfev=5000)
    
    return popt

In [19]:
# The fit_around_px value has to be adjusted to get proper fitting
fit_around_px = 15 #Define the fitting area around xo, yo
nbr_para_fit = 5 #Number of parameters to be fitted

#Np.array with the results from the 2D Gaussian fit
fit_values = np.zeros((point_center.shape[0], nbr_para_fit), dtype=np.float32)
exp_data = np.zeros((2*fit_around_px, 2*fit_around_px), dtype=np.float32)

for i in range(0,point_center.shape[0]-1):
    exp_data=specImg.data[point_center[i,0]-fit_around_px:point_center[i,0]+fit_around_px+1,point_center[i,1]-fit_around_px:point_center[i,1]+fit_around_px+1]
    fit_values[i]=twoD_Gaussian_fit(exp_data)
    i=i+1
    
#Copy of the specImg, where the local min/max will be stored
max_fit_specImg = specImg.deepcopy()
max_fit_specImg.change_dtype('float32')
max_fit_specImg.data[:,:]=0.0

for i in range (0,point_center.shape[0]-1):
    max_fit_specImg.data[int(point_center[i,0]+fit_values[i,2]-fit_around_px),int(point_center[i,1]+fit_values[i,1]-fit_around_px)]=1000.0

/home/martial/anaconda3/lib/python3.5/site-packages/scipy/optimize/minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [31]:
i

1001

In [20]:
max_fit_specImg.plot();

In [21]:
def del_close(max_fit_specImg, close_points, border):
    point_center2=np.zeros((np.size(max_fit_specImg.data.nonzero()[1]),2),dtype=int)
    point_center2[:,:]=0
    x = max_fit_specImg.data.nonzero()[0][:]
    y = max_fit_specImg.data.nonzero()[1][:]
    
    count = 0
    for j in range (0,np.size(x)-1):
        notok = True
        for i in range (j+1,np.size(x)-1):
            d=list_distance(np.array([[x[j],y[j]]]),np.array([[x[i],y[i]]]))[0][0]
            if d<close_points:
                notok = False
                x[i]=(x[j]+x[i])/2
                y[i]=(y[j]+y[i])/2
                break
        if notok == True and x[j] > border and y[j] > border:
            point_center2[count,0]=x[j]
            point_center2[count,1]=y[j]
            count = count +1
            
    point_center3=np.zeros((count,2),dtype=int)
    point_center3[:,:]=point_center2[0:count,:]

    return point_center3

In [22]:
point_center = del_close(max_fit_specImg, 20, 100)

In [23]:
# The fit_around_px value has to be adjusted to get proper fitting
fit_around_px = 20 #Define the fitting area around xo, yo
nbr_para_fit = 5 #Number of parameters to be fitted

#Np.array with the results from the 2D Gaussian fit
fit_values = np.zeros((point_center.shape[0], nbr_para_fit), dtype=np.float32)
fit_values[:] = 0
#exp_data = np.zeros((2*fit_around_px, 2*fit_around_px), dtype=np.float32)

for i in range(0,point_center.shape[0]-1):
    exp_data=specImg.data[point_center[i,0]-fit_around_px:point_center[i,0]+fit_around_px+1,point_center[i,1]-fit_around_px:point_center[i,1]+fit_around_px+1]
    fit_values[i]=twoD_Gaussian_fit(exp_data)
    i=i+1
    
#Copy of the specImg, where the local min/max will be stored
max_fit_specImg = specImg.deepcopy()
max_fit_specImg.change_dtype('float32')
max_fit_specImg.data[:,:]=0.0

for i in range (0,point_center.shape[0]-1):
    max_fit_specImg.data[int(point_center[i,0]+fit_values[i,2]-fit_around_px),int(point_center[i,1]+fit_values[i,1]-fit_around_px)]=1000.0

In [24]:
max_fit_specImg.plot()

Traceback (most recent call last):
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/backends/backend_qt5.py", line 519, in _draw_idle
    self.draw()
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/backends/backend_agg.py", line 433, in draw
    self.figure.draw(self.renderer)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/figure.py", line 1475, in draw
    renderer, self, artists, self.suppressComposite)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/image.py", line 141, in _draw_list_compositing_images
    a.draw(renderer)
  File "/home/martial/anaconda3/lib/python3.5/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/martial/anaconda3/lib/python3.5/site-package

In [ ]:
#max_fit_specImg.plot()
max_fit_specImg.change_dtype('float32')
max_fit_specImg.save(path+'HAADF_9039_crop.rpl')

In [36]:
# Create the image of the fitted Gaussian
fit_around_px=30
exp_data = np.zeros((2*fit_around_px+1, 2*fit_around_px+1), dtype=np.float32)

## Create a new image using the same amplitude and sigma for all atoms
fit_values_copy = fit_values
fit_values_copy[:,0] =1000
fit_values_copy[:,3] =2
fit_values_copy[:,4] =0

x = np.linspace(0, np.shape(exp_data)[0], np.shape(exp_data)[0])
y = np.linspace(0, np.shape(exp_data)[1], np.shape(exp_data)[1])
u = np.meshgrid(x, y)

fit_image_specImg = specImg.deepcopy()
fit_image_specImg.change_dtype('float32')
fit_image_specImg.data[:,:]=0.0

for i in range(0,point_center.shape[0]-1): #
    fit_image_specImg.data[point_center[i,0]-fit_around_px:point_center[i,0]+fit_around_px+1,point_center[i,1]-fit_around_px:point_center[i,1]+fit_around_px+1] = fit_image_specImg.data[point_center[i,0]-fit_around_px:point_center[i,0]+fit_around_px+1,point_center[i,1]-fit_around_px:point_center[i,1]+fit_around_px+1]+twoD_Gaussian(u, fit_values_copy[i])
    i=i+1

In [ ]:
specImg.change_dtype('float32')
specImg.save(path+'HAADF_9039_original.rpl')

In [38]:
fit_image_specImg.plot()

In [39]:
fit_image_specImg.save(path+'HAADF_9039_crop.rpl')

Overwrite '/home/martial/Data/Divers_Project_NTU/Paloma/Test_fitting/HAADF_9039_crop.rpl' (y/n)?
y


In [ ]:
#Plot the difference images between the fitted values and the raw data
plt.imshow(specImg.data-fit_image) #
plt.colorbar()
plt.show()

In [ ]:
#Plot the image ofthe fitted values
plt.imshow(fit_image)
plt.colorbar()
plt.show()

In [ ]:
#Create the image with the values of the maximun
fit_image_max = np.zeros(specImg.data.shape, dtype=np.float32)
fit_image_max[:,:] = np.mean(fit_values[:,0])

for i in range(0,point_center.shape[0]):
    fit_image_max[point_center[i,0]-fit_around_px:point_center[i,0]+fit_around_px+1,point_center[i,1]-fit_around_px:point_center[i,1]+fit_around_px+1] = fit_values[i,0]
    i=i+1

In [ ]:
#Plot an image of the values of the maximum
plt.imshow(fit_image_max)
plt.colorbar()

In [ ]:
plt.hist(fit_values[:,0],bins=20)
plt.show()

In [ ]:
#In case of fit with a non symetric gaussian
def twoD_Gaussian_theta((x, y), amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [ ]:
vor=sc.spatial.Voronoi(point_center, incremental=True)

In [ ]:
sc.spatial.voronoi_plot_2d(vor)

In [ ]:
min_specImg.plot()

In [ ]:
specImg.plot()

In [ ]:
polyarea = np.zeros(len(vor.regions))

for region in range(len(vor.regions)): 
    poly = Polygon((vor.vertices[vor.regions[region]]))
    count_o=0
    count_t=0
    print region "/" len(vor.regions)
    for x, y in it.product(range(0,specImg.data.shape[0]), range(0,specImg.data.shape[1])):
        if poly.contains(Point(x,y)) == True:
            count_o = count_o + specImg.data[x,y]
            count_t = count_t + 1
    polyarea[region] = count_o / count_t

In [ ]:
iL ne faut prendre en compte uniquement les pixels proche du min (3 ou 4)

In [ ]:
plt.hist(polyarea)
plt.show()

In [ ]:
histogram = np.histogram(polyarea, bins=np.arange(min(polyarea), max(polyarea) + 0.005, 0.005))  